# Final Project
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__  
Throughout this course you’ve engaged with key principles required to develop scalable machine learning analyses for structured and unstructured data. Working in Hadoop Streaming and Spark you’ve learned to translate common machine learning algorithms into Map-Reduce style implementations. You’ve developed the ability to evaluate Machine Learning approaches both in terms of their predictive performance as well as their scalability. For the final project you will demonstrate these skills by solving a machine learning challenge on a new dataset. Your job is to perform Click Through Rate prediction on a large dataset of Criteo advertising data made public as part of a Kaggle competition a few years back. As you perform your analysis, keep in mind that we are not grading you on the final performance of your model or how ‘advanced’ the techniques you use but rather on your ability to explain and develop a scalable machine learning approach to answering a real question.

More about the dataset:
https://www.kaggle.com/c/criteo-display-ad-challenge

# Notebook Set-Up
Before starting your homework run the following cells to confirm your setup.

In [54]:
import re
import ast
import time
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml.feature import FeatureHasher
from pyspark.sql import SQLContext, Row
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import SparseVector
from pyspark.sql.types import IntegerType
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import FeatureHasher
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [55]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [56]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [57]:
# start Spark Session
from pyspark.sql import SparkSession
app_name = "fproj_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext
print(sc)

<SparkContext master=yarn appName=pyspark-shell>


In [33]:
#!head -n 1000 data/train.txt > data/train1000.txt
#!tail -n 200 data/train.txt > data/test200.txt

In [34]:
sqlContext = SQLContext(sc)

# Load a text file and convert each line to a Row.
#lines = sc.textFile("data/train.txt")
lines = sc.textFile("gs://midsw261/data/train.txt")
parts = lines.map(lambda l: l.split("\t"))
projectTrain = parts.map(lambda p: Row(label=int(p[0]), I1=p[1], I2=p[2],\
                    I3=p[3], I4=p[4], I5=p[5], I6=p[6],\
                    I7=p[7], I8=p[8], I9=p[9], I10=p[10],\
                    I11=p[11], I12=p[12], I13=p[13], C1=p[14], C2=p[15], C3=p[16],\
                    C4=p[17], C5=p[18], C6=p[19], C7=p[20], C8=p[21], C9=p[22],\
                    C10=p[23], C11=p[24], C12=p[25], C13=p[26], C14=p[27], C15=p[28],\
                    C16=p[29], C17=p[30], C18=p[31], C19=p[32], C20=p[33], C21=p[34],\
                    C22=p[35], C23=p[36], C24=p[37], C25=p[38], C26=p[39]))

# Infer the schema, and register the DataFrame as a table.
fullDF = sqlContext.createDataFrame(projectTrain)
fullDF.registerTempTable("trainTable")

In [58]:
%%time

splits=fullDF.randomSplit([0.7, 0.1, 0.2],2018)
trainDF,valDF,testDF = splits

# Copied code from the cells below - Delete start from here

trainhasher = FeatureHasher(inputCols=["label","I1","I2","I3","I4","I5","I6","I7","I8","I9","I10","I11","I12","C1", "C2", "C3", "C14","C5"],
                       outputCol="features")
#trainDF.select("label","I1","I2","I3","I4","I5","I6","I7","I8","I9","I10","I11","I12","C1", "C2", "C3", "C14","C5").show(2)
featurizedTrain = trainhasher.transform(trainDF)
#featurizedTrain.show(3)
featurizedTrain.select("label","features").show(2,truncate=False)


testhasher = FeatureHasher(inputCols=["label","I1","I2","I3","I4","I5","I6","I7","I8","I9","I10","I11","I12","C1", "C2", "C3", "C14","C5"],
                       outputCol="features")
testDF.select("label","I1","I2","I3","I4","I5","I6","I7","I8","I9","I10","I11","I12","C1", "C2", "C3", "C14","C5").show(2)
featurizedTest = testhasher.transform(testDF)

featurizedTest.select("features").show(2,truncate=False)

lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter = 10)
lrModel = lr.fit(featurizedTrain)
print(lrModel)

predictions = lrModel.transform(featurizedTest)
predictions.show(2)

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="label")
evaluator.evaluate(predictions)
evaluator.getMetricName()
print('Model Intercept: ', lrModel.intercept)
weights = lrModel.coefficients
weights = [(float(w),) for w in weights]  # convert numpy type to float, and to tuple
weightsDF = sqlContext.createDataFrame(weights, ["Feature Weight"])
display(weightsDF)
weightsDF.show(2)
print("Evaluator:")
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="label")
evaluator.evaluate(predictions)
print("Done")

# Delete end

<class 'list'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                                    |
+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |(262144,[6050,41297,48035,79843,88321,90447,114826,138833,169796,176890,183700,186642,199053,207626,212354,222967,239110,252737],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0]) |
|1    |

DataFrame[Feature Weight: double]

+--------------------+
|      Feature Weight|
+--------------------+
|-0.05849690204954...|
| -0.9296982796503523|
+--------------------+
only showing top 2 rows

Evaluator:
Done
CPU times: user 4.72 s, sys: 136 ms, total: 4.86 s
Wall time: 6min 27s


In [52]:
print(type(testhasher))

NameError: name 'testhasher' is not defined

In [39]:
#trainDF = splits[0]
trainDF,valDF,testDF = splits
print(type(trainDF))
print(type(fullDF))

NameError: name 'splits' is not defined

In [20]:
#trainDF.count()
#testDF.count()
#valDF.count()
testDF.show(1)

NameError: name 'testDF' is not defined

In [21]:
trainDF.printSchema()
trainDF.select('label').show(4)

NameError: name 'trainDF' is not defined

In [49]:
# Load a text file and convert each line to a Row.
lines = sc.textFile("data/test200.txt")
parts = lines.map(lambda l: l.split("\t"))
projectTest = parts.map(lambda p: Row(label=int(p[0]), I1=p[1], I2=p[2],\
                    I3=p[3], I4=p[4], I5=p[5], I6=p[6],\
                    I7=p[7], I8=p[8], I9=p[9], I10=p[10],\
                    I11=p[11], I12=p[12], I13=p[13], C1=p[14], C2=p[15], C3=p[16],\
                    C4=p[17], C5=p[18], C6=p[19], C7=p[20], C8=p[21], C9=p[22],\
                    C10=p[23], C11=p[24], C12=p[25], C13=p[26], C14=p[27], C15=p[28],\
                    C16=p[29], C17=p[30], C18=p[31], C19=p[32], C20=p[33], C21=p[34],\
                    C22=p[35], C23=p[36], C24=p[37], C25=p[38], C26=p[39]))

# Infer the schema, and register the DataFrame as a table.
testDF = sqlContext.createDataFrame(projectTest)
testDF.registerTempTable("trainTable")

In [24]:
trainDF.show(1)
testDF.show(1)
valDF.show(1)

NameError: name 'trainDF' is not defined

In [14]:
%%time
trainhasher = FeatureHasher(inputCols=["label","I1","I2","I3","I4","I5","I6","I7","I8","I9","I10","I11","I12","C1", "C2", "C3", "C14","C5"],
                       outputCol="features")
#trainDF.select("label","I1","I2","I3","I4","I5","I6","I7","I8","I9","I10","I11","I12","C1", "C2", "C3", "C14","C5").show(2)
featurizedTrain = trainhasher.transform(trainDF)
#featurizedTrain.show(3)
featurizedTrain.select("label","features").show(2,truncate=False)



NameError: name 'trainDF' is not defined

In [44]:
%%time
testhasher = FeatureHasher(inputCols=["label","I1","I2","I3","I4","I5","I6","I7","I8","I9","I10","I11","I12","C1", "C2", "C3", "C14","C5"],
                       outputCol="features")
testDF.select("label","I1","I2","I3","I4","I5","I6","I7","I8","I9","I10","I11","I12","C1", "C2", "C3", "C14","C5").show(2)
featurizedTest = testhasher.transform(testDF)

featurizedTest.select("features").show(2,truncate=False)

NameError: name 'testDF' is not defined

In [48]:
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter = 10)
lrModel = lr.fit(featurizedTrain)
print(lrModel)

NameError: name 'featurizedTrain' is not defined

In [53]:
predictions = lrModel.transform(featurizedTest)
predictions.show(2)

+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+---+---+---+---+----+---+---+----+---+---+---+---+-----+--------------------+--------------------+--------------------+----------+
|      C1|     C10|     C11|     C12|     C13|     C14|     C15|     C16|     C17|     C18|     C19|      C2|     C20|     C21|     C22|     C23|     C24|     C25|     C26|      C3|      C4|      C5|      C6|      C7|      C8|      C9| I1|I10|I11|I12|I13|  I2| I3| I4|  I5| I6| I7| I8| I9|label|            features|       rawPrediction|         probability|prediction|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+---+---+---+---+----+

In [54]:
predictions.printSchema()
predictions.select("prediction").show(5)



root
 |-- C1: string (nullable = true)
 |-- C10: string (nullable = true)
 |-- C11: string (nullable = true)
 |-- C12: string (nullable = true)
 |-- C13: string (nullable = true)
 |-- C14: string (nullable = true)
 |-- C15: string (nullable = true)
 |-- C16: string (nullable = true)
 |-- C17: string (nullable = true)
 |-- C18: string (nullable = true)
 |-- C19: string (nullable = true)
 |-- C2: string (nullable = true)
 |-- C20: string (nullable = true)
 |-- C21: string (nullable = true)
 |-- C22: string (nullable = true)
 |-- C23: string (nullable = true)
 |-- C24: string (nullable = true)
 |-- C25: string (nullable = true)
 |-- C26: string (nullable = true)
 |-- C3: string (nullable = true)
 |-- C4: string (nullable = true)
 |-- C5: string (nullable = true)
 |-- C6: string (nullable = true)
 |-- C7: string (nullable = true)
 |-- C8: string (nullable = true)
 |-- C9: string (nullable = true)
 |-- I1: string (nullable = true)
 |-- I10: string (nullable = true)
 |-- I11: string (nullabl

In [55]:
predictions.show(3)

+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+---+---+---+---+----+---+---+----+---+---+---+---+-----+--------------------+--------------------+--------------------+----------+
|      C1|     C10|     C11|     C12|     C13|     C14|     C15|     C16|     C17|     C18|     C19|      C2|     C20|     C21|     C22|     C23|     C24|     C25|     C26|      C3|      C4|      C5|      C6|      C7|      C8|      C9| I1|I10|I11|I12|I13|  I2| I3| I4|  I5| I6| I7| I8| I9|label|            features|       rawPrediction|         probability|prediction|
+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---+---+---+---+---+----+

In [58]:
#print(lr.explainParams())
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="label")
evaluator.evaluate(predictions)
evaluator.getMetricName()
print('Model Intercept: ', lrModel.intercept)
weights = lrModel.coefficients
weights = [(float(w),) for w in weights]  # convert numpy type to float, and to tuple
weightsDF = sqlContext.createDataFrame(weights, ["Feature Weight"])
display(weightsDF)
weightsDF.show(2)

Model Intercept:  -1.539475798610839


DataFrame[Feature Weight: double]

+--------------+
|Feature Weight|
+--------------+
|           0.0|
|           0.0|
+--------------+
only showing top 2 rows



In [53]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="label")
evaluator.evaluate(predictions)

NameError: name 'predictions' is not defined